In [2]:
import torch


In [4]:
print(torch.cuda.is_available())
print(torch.version.cuda)


True
12.4


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from PIL import Image
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

mask_dataset_path = r"C:\Users\bhatt\OneDrive\Desktop\math-helper\DataSet\Mask"
no_mask_dataset_path = r"C:\Users\bhatt\OneDrive\Desktop\math-helper\DataSet\No-Mask"

def load_images_from_folder(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path).convert("RGB")
                img = transform(img)
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return images, labels

mask_images, mask_labels = load_images_from_folder(mask_dataset_path, label=1)
no_mask_images, no_mask_labels = load_images_from_folder(no_mask_dataset_path, label=0)

all_images = mask_images + no_mask_images
all_labels = mask_labels + no_mask_labels

images_tensor = torch.stack(all_images)
labels_tensor = torch.tensor(all_labels)

dataset = TensorDataset(images_tensor, labels_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = x.view(-1, 512 * 3 * 3)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

model_save_path = 'modelv5.pth'
torch.save(model.state_dict(), model_save_path)


Epoch [1/50], Loss: 0.4844
Epoch [2/50], Loss: 0.1758
Epoch [3/50], Loss: 0.0755
Epoch [4/50], Loss: 0.0506
Epoch [5/50], Loss: 0.0294
Epoch [6/50], Loss: 0.0198
Epoch [7/50], Loss: 0.0184
Epoch [8/50], Loss: 0.0216
Epoch [9/50], Loss: 0.0134
Epoch [10/50], Loss: 0.0106
Epoch [11/50], Loss: 0.0144
Epoch [12/50], Loss: 0.0105
Epoch [13/50], Loss: 0.0105
Epoch [14/50], Loss: 0.0112
Epoch [15/50], Loss: 0.0113
Epoch [16/50], Loss: 0.0098
Epoch [17/50], Loss: 0.0149
Epoch [18/50], Loss: 0.0109
Epoch [19/50], Loss: 0.0121
Epoch [20/50], Loss: 0.0120
Epoch [21/50], Loss: 0.0114
Epoch [22/50], Loss: 0.0097
Epoch [23/50], Loss: 0.0086
Epoch [24/50], Loss: 0.0167
Epoch [25/50], Loss: 0.0134
Epoch [26/50], Loss: 0.0140
Epoch [27/50], Loss: 0.0094
Epoch [28/50], Loss: 0.0130
Epoch [29/50], Loss: 0.0099
Epoch [30/50], Loss: 0.0104
Epoch [31/50], Loss: 0.0139
Epoch [32/50], Loss: 0.0122
Epoch [33/50], Loss: 0.0138
Epoch [34/50], Loss: 0.0147
Epoch [35/50], Loss: 0.0123
Epoch [36/50], Loss: 0.0133
E

In [24]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define the CNN model with Batch Normalization and Dropout (same as training)
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = x.view(-1, 512 * 3 * 3)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# Load the model
model = CNNModel().to(device)
model.load_state_dict(torch.load('modelv3.pth'))
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Convert to RGB
    image = transform(image)  # Apply the transform
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to GPU
    return image

# Predict function
def predict_image(image_path, model):
    image = load_and_preprocess_image(image_path)

    # Make prediction
    with torch.no_grad():
        prediction = model(image)
        predicted_class = torch.argmax(prediction, dim=1).item()

    # Check the prediction result
    if predicted_class == 1:
        result = "Has a mask"
    else:
        result = "No mask"

    print(f"Prediction: {prediction}, Result: {result}")

# Specify the image path directly in the script
new_test_image_path = "m.jpg"  # Replace with your image path

# Predict for the provided image
predict_image(new_test_image_path, model)


Using device: cuda
Prediction: tensor([[0.1481, 0.2342]], device='cuda:0'), Result: Has a mask


C:\Users\bhatt\AppData\Local\Temp\ipykernel_24180\2480407688.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modelv3.pth'))


: 

In [13]:
import cv2
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = x.view(-1, 512 * 3 * 3)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

model = CNNModel().to(device)
model.load_state_dict(torch.load('modelv3.pth'))
model.eval()

transform = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def preprocess_face(face_image):
    face_image = Image.fromarray(face_image).convert('RGB')
    face_image = transform(face_image)
    face_image = face_image.unsqueeze(0).to(device)
    return face_image

def predict_mask(face_image, model):
    face_tensor = preprocess_face(face_image)
    with torch.no_grad():
        prediction = model(face_tensor)
        predicted_class = torch.argmax(prediction, dim=1).item()
    return "Mask" if predicted_class == 1 else "No Mask"

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        result = predict_mask(face, model)
        color = (0, 255, 0) if result == "Mask" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, result, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow('Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using device: cuda


C:\Users\bhatt\AppData\Local\Temp\ipykernel_24180\3608040236.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modelv3.pth'))
